# Python coding bootcamp - Notebook 4

1. Combining data
2. Statistical analysis
3. The split-apply-combine strategy
4. Time series

&copy; Francis WOLINSKI 2024

<div class="alert alert-danger">
    <h3><i class="fa fa-plus-square"></i>  PART 1</h3>
</div>

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns

# display options
pd.set_option("display.min_rows", 16)
pd.set_option("display.max_columns", 30)

#### Dataset 1

<h3><i class="fa fa-database"></i></h3>

The GeoNames geographical database covers all countries and contains over eleven million placenames that are available for download free of charge.

Source: http://download.geonames.org/export/dump/

File : <code>cities500.zip</code>

All cities with a population > 500 or seats of adm div down to PPLA4 (ca 200,000), see 'geoname' table for columns.

In [ ]:
# load dataset 1
df = pd.read_csv('data/cities500.zip',
                   sep='\t',
                   header=None,
                   names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class', 'feature code', 'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification date'],
                   dtype={'admin1 code': str, 'admin2 code': str, 'admin3 code': str, 'admin4 code': str})
df.shape

In [ ]:
# head
df.head()

In [ ]:
# check columns with NaN
df.isna().any(axis=0)

In [ ]:
# which country code is NaN?
df.loc[df['country code'].isna()].head()

We reload the dataset with the appropriate option for dealing with the `NA` code which represents Namibia and NA values.

In [ ]:
# reload dataset 1
# NA values are only empty strings and -9999 numbers
df = pd.read_csv('data/cities500.zip',
                   sep='\t',
                   header=None,
                   names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class', 'feature code', 'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification date'],
                   dtype={'admin1 code': str, 'admin2 code': str, 'admin3 code': str, 'admin4 code': str},
                   keep_default_na=False,
                   na_values=['', -9999])

df.isna().any(axis=0)

In [ ]:
# number of unique values in each column
df.nunique()

#### Dataset 2

<h3><i class="fa fa-database"></i></h3>

Country Codes

Source: https://www.geonames.org/countries/

The `read_html()` pandas function is able to retrieve all tables from an HTML page. It returns a list of `DataFrame` objects.

In [ ]:
# require internet access and lxml
if False:
    var = pd.read_html('https://www.geonames.org/countries/',
                       header=0,
                       keep_default_na=False,  # so that "NA" which stands for North America is not discarded
                       encoding='utf-8'
                      )

    # display the shapes of the found tables
    print([x.shape for x in var])
    # get the country DataFrame
    df_country = var[1]
else:
    df_country = pd.read_csv('data/GeoNames.csv', keep_default_na=False)
    
df_country.shape

One can see that the `NA` code is used for a country (Namibia) but also for a continent (North America).

There are 7 continents in the dataset: Africa, Europe, Asia, North America, Oceania, South America and Antartica.

In [ ]:
# number of countries by continent 
df_country['Continent'].value_counts()

In [ ]:
# which countries are in AN continent?
df_country.loc[df_country['Continent'] == 'AN', 'Country']

# 1. Combining data

The `pandas` library provides functions and methods to combine data.

## 1.1 Concatenation

It is possible to concatenate several `Series`or `DataFrame` objects:
- The `concat()` function takes a list of `Series`or `DataFrame` objects
- The option `axis=` can be used to specify whether the concatenation is to be performed vertically or horizontally.
- The `concat()` function has already been encountered for concatenating all the US name files (see notebook `python_coding_bootcamp_2`).

## 1.2 Merging

It is possible to merge 2 `DataFrame` objects by performing a database-style join operation by columns or indexes.
- The `merge()` function takes 2 `DataFrame` objects.

The database-style join operation requires a column of each `DataFrame` object on which a join will be perfomed and is specified with the `how`option:

Merge method | SQL Join Name | Description
-|-|-
inner | INNER JOIN | Use intersection of keys from both frames
left | LEFT OUTER JOIN | Use keys from left frame only
right | RIGHT OUTER JOIN | Use keys from right frame only
outer | FULL OUTER JOIN | Use union of keys from both frames
cross | CROSS JOIN | Cartesian product of both frames

By default the join is an `inner` join.

As a result, we have a new `DataFrame` with 28 columns: 19 from the cities `DataFrame` and 9 from the countries `DataFrame`. On each line the values of the `country code` and `ISO-3166 alpha2` columns are identical.

**Warning**: in the merged `DataFrame` there are 2 columns dealing with population: the column `population` refers to the population of cities and the column `Population` refers to the population of countries. If the column names had been identical, pandas would have added suffixes to their names.

Merging and joining 2 `DataFrame` objects have many options.

In [ ]:
# merge df and df_countries by iso2
# run this cell only once
df = pd.merge(df,
              df_country,
              left_on='country code',
              right_on='ISO-3166 alpha2',
              how='left')

df.to_pickle('df_cities_countries.pkl')
df.head()

In [ ]:
# check the merge: unnecessary of course !
(df['country code'] == df['ISO-3166 alpha2']).all()

The number of cities by continent.

In [ ]:
# number of cities by continent
df['Continent'].value_counts()

<div class="alert alert-warning">
    <h3><i class="fa fa-book"></i> Further reading</h3>
    <ul>
        <li><a href='https://pandas.pydata.org/docs/user_guide/merging.html'>Merge, join, concatenate and compare</a></li>
    </ul>
</div>

# 2. Statistical analysis

## 2.1 The `describe()` method

The `describe()` method applied to `Series` or to `DataFrame` objects returns 8 statistical indicators on numerical columns: count, mean, standard deviation, quartiles (25%, 50%, 75%), minimum and maximum.

The `describe()` method applied to `Series` objects returns 4 statistical indicators on non numerical dtypes: count, unique, top (most common value), and freq (most common value's frequency).

The outpout is an object of same type.

This method is useful to get a quick overview of a new dataset.

In [ ]:
# describe for numerical Series
df['population'].describe()

In [ ]:
# describe for non numerical Series
df['country code'].describe()

In [ ]:
# describe for DataFrame
df.describe()

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 1 &starf;</h3>
    <ul>
        <li>What is the most used city name?</li>
    </ul>
</div>

In [ ]:
# %load notebook4/ex_01.py

## 2.2 Statistical methods

A full set of statistical methods exist and most of them have already been introduced.

- count: `count()`
- sum: `sum()`
- mean: `mean()`
- median: `median()`
- minimum: `min()`
- maximum: `max()`
- product: `prod()`
- percent change: `pct_change()`
- ranking: `rank()`
- standard deviation: `std()`
- variance: `var()`
- covariance: `cov()`
- correlation: `corr()`
- quantile: `quantile()`
- cummulative sum: `cumsum()`
- cummulative product: `cumprod()`
- cummulative minimum: `cummin()`
- cummulative maximum: `cummax()`
- label of minimum: `idxmin()`
- label of maximum: `idxmax()`
- sample: `sample()`
- clipping: `clip()`

<div class="alert alert-warning">
    <h3><i class="fa fa-book"></i> Further reading</h3>
    <ul>
        <li><a href='https://pandas.pydata.org/pandas-docs/stable/reference/frame.html#computations-descriptive-stats'>Computations / descriptive stats</a></li>
    </ul>
</div>

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 2 &starf;&starf;</h3>
    <ul>
        <li>Which city has the largest population?</li>
        <li>Which city has the lowest elevation?</li>
        <li>Which city has the highest elevation?</li>
        <li>Which city has the largest number of alternative names? How many names? What are the names?</li>
    </ul>
</div>

In [ ]:
# %load notebook4/ex_02.py

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 3 &starf;&starf;&starf;</h3>
    <ul>
        <li>Implement a function which finds the closest city given some decimal coordinates (latitude, longitude).</li>
        <li>Which city is closest to the crossing between the Equator and the Greenwich Meridian (latitude and longitude are zero)?</li>
        <li>Implement a function which computes the decimal latitude or longitude given them in degrees, minutes and seconds.</li>
        <li>Pick up a city, check its coordinates in <a href="https://en.wikipedia.org">Wikipedia</a>, then use the combined functions to search it in the dataset.</li>
    </ul>
</div>

In [ ]:
# %load notebook4/ex_03.py

In [ ]:
# Which city is closest to a given position?

def get_city(lat, long):
    pass

get_city(0.0, 0.0)

In [ ]:
# coordinate to decimal
def dms2gps(d, m, s):
    pass

In [ ]:
# test with any world city
# north/east = positive, south/west = negative
# Area 51: 37° 14′ 06″ north, 115° 48′ 40″ west
# see https://fr.wikipedia.org/wiki/Zone_51
get_city(dms2gps(37, 14, 6), dms2gps(-115, -48, -40))

## 2.3 Discretization of numerical values

### 2.3.1 The `cut()` function

The `cut()` function discretizes a `Series` object of numerical values according to thresholds and by mapping to categorical values.

It returns a `Series` object which shares the same index of the initial one and whose values dtype is categorical and ordered along with the given thresholds.

By default the bins include the rightmost edge. This can be changed by using the option `right=False`.

In [ ]:
# arbitrary discretization of populations in:
# - small [0-1,000]
# - medium [1,000-10,000]
# - large [10,000-100,000]
# - big [100,000-1,000,000]
# - huge [1,000,000-infinity]
var1 = pd.cut(df["population"],
               [0, 1000, 10000, 100000, 1000000, np.inf],
               labels=["small", "medium", "large", "big", "huge"])
var1

Note the use of `np.inf`, a special `numpy` constant to denote $+\infty$. One can use either `-np.inf` or `np.NINF` to denote $-\infty$.

In [ ]:
# value counts of var1
var1.value_counts()

Countplot with seaborn.

In [ ]:
# countplot with var1
sns.countplot(x=var1);

Countplot with log scale with seaborn.

In [ ]:
# countplot with var1 and a log scale
ax = sns.countplot(x=var1)
ax.set_yscale('log');

### The `qcut()` function

The `qcut()` function is a quantile-based discretization function. It discretizes a variable into equal-sized buckets based on rank or based on sample quantiles.

It returns a `Series` object which shares the same index the the initial one and whose values dtype is categorical and ordered along with the given labels.

By using the option `retbins=True`, the function returns also the array of the different thresholds. The length of the array is +1 than the number of buckets since the last value corresponds to the maximum value.

In [ ]:
# arbitrary discretization of populations in 5 "identical" buckets: A, B, C, D and E
var2 = pd.qcut(df["population"],
               5,
               labels=["E", "D", "C", "B", "A"])
var2

In [ ]:
# value counts of var2
var2.value_counts(sort=False)

In [ ]:
# countplot with var2
sns.countplot(x=var2);

By default the labels are the bins which define the different buckets.

In [ ]:
# no label given
var3 = pd.qcut(df["population"], 5)
var3

In [ ]:
# value counts
var3.value_counts(sort=False)

In [ ]:
# option retbins=True
_, var4 = pd.qcut(df["population"], 5, retbins=True)
var4

<div class="alert alert-danger">
    <h3><i class="fa fa-plus-square"></i>  PART 2</h3>
</div>

In [ ]:
# reload df
df = pd.read_pickle('df_cities_countries.pkl')
df.shape

# 3. The split-apply-combine strategy

## 3.1 Introduction

The **split-apply-combine strategy** consists in:
- Splitting the data into groups based on some criteria.
- Applying a function to each group independently.
- Combining the results into a data structure.

The `groupby()` method splits the data according to identical values of one or several columns (like a pivot table). It returns a `DataFrameGroupBy` object which can be viewed like a dictionary whose:
- keys are the modalities of the column(s) used to split the data
- and values are subsets of the initial `DataFrame` object, and are either `Series` or `DataFrame` objects.

A function (generally an aggregation function) is then applied to each value of the `DataFrameGroupBy` object.

The result is finally produced by the combination (or concatenation) of the different values output by the function.

This method is similar with the `MapReduce` used in big data architecture.

## 3.2 Standard methods

Several methods apply to `DataFrameGroupBy` object:

- `size()`: Compute group sizes
- `count()`: Compute count of group
- `mean()`: Compute mean of groups
- `sum()`: Compute sum of group values
- `std()`: Standard deviation of groups
- `var()`: Compute variance of groups
- `describe()`: Generates descriptive statistics
- `first()`: Compute first of group values
- `last()`: Compute last of group values
- `nth()`: Take nth value
- `min()`: Compute min of group values
- `max()`: Compute max of group values

The `size()` method returns a `Series` object with the size of each group.

In [ ]:
# size of groups
df.groupby('Continent').size()

In [ ]:
# we can check that sum of sizes is identical to the length of dataframe
df.groupby('Continent').size().sum() == len(df)

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 4 &starf;</h3>
    <ul>
        <li>Perform a groupby with the column "Country".</li>
        <li>Which countries get the maximum and the minimum group size?</li>
    </ul>
</div>

In [ ]:
# %load notebook4/ex_04.py

The `describe()` method also applies to `DataFrameGroupBy` objects. It returns the standard statistics on each numerical column of each group.

In [ ]:
# describe groups
df.groupby('Continent').describe()

The `sum()` method computes the sum of each numerical column for each group.

In [ ]:
df.groupby('Continent').sum(numeric_only=True)

## 3.3 The `aggregate()` method

The `aggregate()`, or `agg()`, method aggregates all values obtained from groups by passing one, or several function(s), or again different functions by columns. Note that the functions might be used by their name, or taken from the `numpy` module, or again be user-defined.

### 3.3.1 Aggregate with a single function

When passing a single function, the output is a `DataFrame` or a `DataFrame` object whose columns are those of the initial `DataFrame` object to which the function is valid.

In [ ]:
# valid for all columns
df.groupby('Continent').agg('mean', numeric_only=True)  # same as mean(numeric_only=True)

### 3.3.2 Aggregate with a list of functions

When passing a list of functions, the output is a `DataFrame` object with hierarchical columns (see below): the first level shows the columns of the initial `DataFrame` object and the second one the name of the functions applied to all column values.

In [ ]:
# passing a list of functions
df.groupby('Continent').agg(['mean', 'std'], numeric_only=True)

### 3.3.2 Aggregate with a dict of functions

When passing a dictionary of functions whose keys are column names of the initial `DataFrame` object, the output is also a `DataFrame` object with hierarchical columns: the first level shows the keys and the second one the name of the functions applied to each column values.

In [ ]:
# passing a dict of functions
df.groupby('Continent').agg({'population': 'sum',
                             'elevation': 'mean',
                             'Country': ['min', 'max']},
                            numeric_only=True)

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 5 &starf;</h3>
    <ul>
        <li>Compute the minimum and maximum of longitude and latitude by continent.</li>
    </ul>
</div>

In [ ]:
# %load notebook4/ex_05.py

## 3.4 The `apply()` method

The `apply()` method enables to apply any method to each group. The results will then be combined in a single data structure.

For instance, we will compute the city with the largest population by continent.

In [ ]:
# return the name with the largest population
def top1city(group):
    return group.loc[group['population'].idxmax(), 'name']

In [ ]:
# use with the whole dataframe
top1city(df)

In [ ]:
# apply on group
df.groupby('Continent').apply(top1city)

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 6 &starf;&starf;</h3>
    <ul>
        <li>Get the top 1 city by country</li>
        <li>Get the top 3 cities by continent</li>
        <li>Get the top 3 cities and population by continent</li>
        <li><b>Hint</b>: If you want to get rid off the extra index, use <code>.droplevel(1)</code> afterwards.
    </ul>
</div>

In [ ]:
# %load notebook4/ex_06.py

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 7 &starf;&starf;&starf;</h3>
    <ul>
        <li>Create a function which computes the mean number of alternate names of each city.</li>
        <li>Test it on the whole dataset.</li>
        <li>Apply this function to the groupby "feature code".</li>
        <li>Which "feature code" gets the largest value?</li>
    </ul>
</div>

In [ ]:
# %load notebook4/ex_07.py

<div class="alert alert-warning">
    <h3><i class="fa fa-book"></i> Further reading</h3>
    <ul>
        <li><a href='https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html'>Group by: split-apply-combine</a></li>
    </ul>
</div>

# 4. Time series


This part is a short introduction to time series management with Python. The subject is indeed a complex one.

The `datetime` standard module enables to manipulate temporal data with Python.

Different modules intend to manage temporal data (`calendar`, `dateutil`, `pytz`, ...); although the `arrow` module intends to unify all those libraries (out of scope).

The `datetime` module provides different temporal data types:
- **date**: date (year, month, day) in the Gregorian calendar
- **time**: time not attached to a date (hours, minutes, seconds, microseconds)
- **datetime**: timestamp (date + time)
- **timedelta**: duration, difference between 2 dates or 2 times (days, hours, minutes, seconds, microseconds)
- **tzinfo**: management of time zones

In [ ]:
# import datetime library
import datetime

## 5.1 Temporal objects

#### Date

In [ ]:
# date
today = datetime.date.today()
today

#### Datetime

In [ ]:
# datetime
now = datetime.datetime.now()
now

In [ ]:
# UTC datetime
now_utc = datetime.datetime.utcnow()
now_utc

#### Time delta

A `timedelta` object represents a duration which might be expressed in days, hours, minutes, seconds and microseconds.

A difference between 2 dates returns a `timedelta` object and conversely the sum or the difference between a `date` object and a `timedelta` object returns a `date` object.

In [ ]:
# difference between 2 dates
today - datetime.date(2024, 1, 1)

In [ ]:
# sum of a date and a timedelta
now + datetime.timedelta(days=100)

In [ ]:
# difference between a date and a timedelta
now - datetime.timedelta(days=100)

#### Time zone

The `pytz` module implements timezone.

A `datetime` object can be converted with the method ` astimezone()` to another `datetime` object within another timezone.

As manipulating timezones increase some complexity, the preferred way of dealing with times is to always work in UTC, converting to localtime only when generating output to be read by humans.

In [ ]:
# import pytz
from pytz import timezone

In [ ]:
# build a timezone object
tz = timezone('Asia/Shanghai')
tz

In [ ]:
# convert a datetime by using a timezone object
now.astimezone(tz)

## 5.2 Writing and reading

The `strftime()` method and the `strptime()` function enable to write and to read dates in and from several formats with a codification borrowed to the C language.

The same codification is used to format an existing date into a string and to build a temporal object from a string in a given format.

The number of possible directives unveils the complexity of the subject.

Directives | Comments
- | -
%a | Day of the week abbreviated
%A | Day of the week
%w | Day of the week 0 = Sunday ... 6 = Saturday
%d | Day of month on 2 digits 01, 02, ..., 31
%j | Day of year on 3 digits 001, 002, ..., 366
%b | Month abbreviated
%B | Month name
%m | Month on 2 digits 01, 02, ..., 12
%U | Number of week in year (Sunday = first day)
%W | Number of week in year (Monday = first day)
%y | Year without the century on 2 digits 00, 01, ..., 99	 
%Y | Year with the century on 4 digits 0001, 0002, ..., 2018, 2019, ..., 9998, 9999
%H | Hour over 24 00, 01, ..., 23
%I | Hour over 12 01, 02, ..., 12
%p | AM or PM
%M | Minute on 2 digits 00, 01, ..., 59
%S | Second on 2 digits 00, 01, ..., 59
%f | Microsecond on 6 digits 000000, 000001, ..., 999999
%z | UTC offset +HHMM or -HHMM
%Z | Time zone 
%c | Representation date and temps
%x | Representation date
%X | Representation time
%% | Character %

In [ ]:
# datetime
now = datetime.datetime.now()
now

In [ ]:
# day of week
now.strftime("%A %d/%m/%Y")

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 8 &starf;</h3>
    <ul>
    <li>Print the current date in the format: "YYYY-MM-DD/HH:MM:SS"</li>
    </ul>
</div>

In [ ]:
# %load notebook4/ex_08.py

The `locale` module helps to manage strings in different languages to get days and months names.

You can check the ISO 639-1 codes for languages: https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes

In [ ]:
# managing locale: try to put your own language
# sometimes for instance 'fr' is not working, try 'fr_FR' or 'fr_FR.UTF-8'
# you can also try with the ISO 639-2 code (with 3 letters)
# this depends on your machine not on Python

import locale

#locale.setlocale(locale.LC_ALL, 'am') # armenian
#locale.setlocale(locale.LC_ALL, 'ar') # arabic
#locale.setlocale(locale.LC_ALL, 'az') # azeri
#locale.setlocale(locale.LC_ALL, 'bn') # bengali
#locale.setlocale(locale.LC_ALL, 'da') # danish
#locale.setlocale(locale.LC_ALL, 'de') # german
#locale.setlocale(locale.LC_ALL, 'es') # spanish
#locale.setlocale(locale.LC_ALL, 'en') # english
#locale.setlocale(locale.LC_ALL, 'fa') # farsi
locale.setlocale(locale.LC_ALL, 'fr') # french
#locale.setlocale(locale.LC_ALL, 'ga') # gaelic
#locale.setlocale(locale.LC_ALL, 'hi') # hindi
#locale.setlocale(locale.LC_ALL, 'he') # hebrew
#locale.setlocale(locale.LC_ALL, 'hr') # croatian
#locale.setlocale(locale.LC_ALL, 'ig') # igbo
#locale.setlocale(locale.LC_ALL, 'it') # italian
#locale.setlocale(locale.LC_ALL, 'ja') # japanese
#locale.setlocale(locale.LC_ALL, 'lt') # lithuanian 
#locale.setlocale(locale.LC_ALL, 'ko') # korean
#locale.setlocale(locale.LC_ALL, 'nl') # dutch
#locale.setlocale(locale.LC_ALL, 'no') # norvegian
#locale.setlocale(locale.LC_ALL, 'pt') # portugese
#locale.setlocale(locale.LC_ALL, 'ro') # romanian
#locale.setlocale(locale.LC_ALL, 'ru') # russian
#locale.setlocale(locale.LC_ALL, 'sq') # albanian
#locale.setlocale(locale.LC_ALL, 'sr') # serbian
#locale.setlocale(locale.LC_ALL, 'th') # thai
#locale.setlocale(locale.LC_ALL, 'tr') # turkish
#locale.setlocale(locale.LC_ALL, 'ukr') # ukrainian
#locale.setlocale(locale.LC_ALL, 'vi') # vietnamian
#locale.setlocale(locale.LC_ALL, 'zh') # chinese
now.strftime("%A %B %d/%m/%Y")

In [ ]:
# reset locale
locale.setlocale(locale.LC_ALL, 'en')

Conversely, the `strptime()` function takes as arguments a string to decode and another string specifying the format of date end returns a `datetime` object.

In [ ]:
# reading from format: AA-MM-DD-HH-MM
# writing to format : HH:MM DD/MM/YYYY
var = datetime.datetime.strptime("19-03-01-12-00", "%y-%m-%d-%H-%M")
var.strftime("%H:%M %d/%m/%Y")

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 9 &starf;</h3>
    <ul>
        <li>Read a timestamp from the ISO 8601 format: "YYYY-MM-DDTHH:MM:SS", e.g., "2024-09-11T15:00:00"</li>
    </ul>
</div>

In [ ]:
# %load notebook4/ex_09.py

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 10 &starf;&starf;&starf;</h3>
    <ul>
        <li>Load the "cities500.zip" DataFrame object, how many timezones do we have?</li>
        <li>Switch the "timezone" column values to "timezone" objects.</li>
        <li>Take the current "datetime" object (now), convert it with all timezones from the "timezone" column, and then produce strings using the format: "'%Y-%m-%d %H:%M:%S'"</li>
        <li>There are 24 hours in a day, how many different times do we get?</li>
</ul>
</div>

In [ ]:
# dataset 1
# load cities
df = pd.read_csv('data/cities500.zip',
                   sep='\t',
                   header=None,
                   names=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class', 'feature code', 'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification date'],
                   dtype={'admin1 code': str, 'admin2 code': str, 'admin3 code': str, 'admin4 code': str},
                   keep_default_na=False,
                   na_values=['', -9999])
df.head()

In [ ]:
# %load notebook4/ex_10.py

## 5.3 Time series

The `pandas` library is able to manage time series.

Time series are a huge subject in itself. This paragraph presents only the main concepts:

- reading time series
- accessing to time series
- resampling time series (upsampling)
- graphics and rolling windows

The `to_datetime()` pandas function works like the `strptime()` function in order to convert a string into a date according to a specific format.

#### Dataset 6

<h3><i class="fa fa-database"></i></h3>

Exchange rates :
- The first column collect the dates
- The following columns collect the exchange rate to EUR by currency

In [ ]:
# dataset with exchange rates
# source: https://www.banque-france.fr

# load the file
exchange_rates = pd.read_csv("data/Webstat_Export.zip",
                        sep=";",
                        na_values='-',
                        decimal=',',
                        skiprows=[1, 2],
                        usecols=range(43),
                        converters={0: lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='ignore')})
exchange_rates

In [ ]:
# extracting currency codes
cols = pd.Series(exchange_rates.columns.tolist()).str.extract('\(([A-Z]{3})\)', expand=False)
exchange_rates.columns = ['Date'] + list(cols[1:])

# selecting a few currencies
currencies = ['USD', 'CHF', 'GBP', 'JPY', 'BRL', 'CNY']
exchange_rates = exchange_rates[['Date'] + currencies]

# drop na
exchange_rates = exchange_rates.dropna()

exchange_rates

### 5.3.1 The `dt` accessor

The `dt` accessor behaves similarly to the `str` accessor for `Series` objects. It enables to access to element-wise datetime attributes.

In [ ]:
# access to year
exchange_rates['Date'].dt.year

In [ ]:
# access to month
exchange_rates['Date'].dt.month

In [ ]:
# access to week days
exchange_rates['Date'].dt.weekday

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 11 &starf;</h3>
    <ul>
        <li>On which days of week currency rates are issued?</li>
    </ul>
</div>

In [ ]:
# %load notebook4/ex_11.py

### 5.3.2 Selecting temporal data

When the index is a datetime object, it is possible to select time range from string representations of dates. It works also for slice selection.

In [ ]:
# setting the index as a datetime object
exchange_rates = exchange_rates.set_index('Date')
exchange_rates = exchange_rates.sort_index()
exchange_rates

Select a full year.

In [ ]:
# yearly data
exchange_rates.loc['2023']

Select a full month in a given year.

In [ ]:
# monthly data
exchange_rates.loc['2023/01'] # or exchange_rates['01/2023']

Slice selection includes the rightmost period.

In [ ]:
# slice selection
exchange_rates['12/2023':'01/2024']

### 5.3.3 Temporal aggregations

It is possible to aggregate data according to a certain period of time by using the `resample()` method with a symbol meaning the period and then to apply an aggregation method.

The `resample()` method behaves similarly to the `groupby()` method. It has been adapted to temporal grouping.

In [ ]:
# yearly aggregation
exchange_rates.resample('A').size()

In [ ]:
# yearly aggregation
exchange_rates.resample('A').mean()

In [ ]:
# monthly aggregation
exchange_rates.resample('M').mean()

#### Symbolic frequencies which can be used (extract)

Here again, the number of possible directives unveils the complexity of the subject...

Alias | Offset type | Description
- | - | -
D | Day | Calendar daily
B | BusinessDay | Business daily
H | Hour | Hourly
T or min | Minute | Minutely
S | Second | Secondly
L or ms | Milli | Millisecond (1/1000th of 1 second)
U | Micro | Microsecond (1/1000000th of 1 second)
M | MonthEnd | Last calendar day of month
BM | BusinessMonthEnd | Last business day (weekday) of month
MS | MonthBegin | First calendar day of month
BMS | BusinessMonthBegin | First weekday of month
W-MON, W-TUE, ... | Week | Weekly on given day of week: MON, TUE, WED, THU, FRI, SAT, or SUN.
Q-JAN, Q-FEB, ... | QuarterEnd | Quarterly dates anchored on last calendar day of each month,for year ending in indicated month: JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, or DEC.
A-JAN, A-FEB, ... | YearEnd | Annual dates anchored on last calendar day of given month: JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, NOV, or DEC.

Source: Python for Data Analysis, Wes McKinney, O'Reilly

### 5.3.4 Temporal graphics

It is possible to display directly temporal graphics.

In [ ]:
# exchange rates
exchange_rates.plot();

In the graphics below, we divide the exchange rates by their respective average to adjust all rates to the same scale.

In [ ]:
# exchange rates divided by their average
(exchange_rates / exchange_rates.mean()).plot();

The `rolling()` function enables to display graphics with a moving average for instance.

In [ ]:
# exchange rates divided by their average with a moving average of 30 days
(exchange_rates / exchange_rates.mean()).rolling(30).mean().plot();

<div class="alert alert-success">
    <h3><i class="fa fa-edit"></i>  Exercise 12 &starf;&starf;</h3>
    <ul>
        <li>Display a chart with exchange rates divided by the last known values</li>
        <li>Display a chart with exchange rates divided by their means with a moving maximum of 100 days</li>
    </ul>
</div>

In [ ]:
# %load notebook4/ex_12.py

# exchange rates divided by their means with a moving maximum of 100 days
(exchange_rates / exchange_rates.iloc[-1]).plot();

# exchange rates divided by their means with a moving maximum of 100 days
(exchange_rates / exchange_rates.mean()).rolling(100).max().plot();
